# Automatic algorithm/strategy selection
Here we will show you how you can perform an algorithm selection, helping you choose the best optimization algorithm for your specific problem.

In [15]:
from metaheuristic_designer import simple
from metaheuristic_designer.operators import OperatorReal
from metaheuristic_designer.selectionMethods import ParentSelection, SurvivorSelection
from metaheuristic_designer.algorithms import AlgorithmSelection, StrategySelection
from metaheuristic_designer.initializers import UniformVectorInitializer
from metaheuristic_designer.strategies import HillClimb, SA, ES, GA, DE, PSO, RandomSearch
from metaheuristic_designer.benchmarks import *
import numpy as np

For the following examples, we will use the Rastrigin function for which we need to find the minimum value. This is a very hard optimization problem and will allow us to compare the performance of different algorithms.

In [16]:
rastrigin_func = Rastrigin(size=2)

## Algorithm selection
We will first explore the AlgorithmSelection class, which launches each of the specified algorithms multiple times and gets some statistics about the evaluation.

We begin by defining the algorithm we are going to use. The ```simple``` package gives us premade implementations of well known optimization algorithms with a single function call, although customization is limited.

In [17]:
# The parameters common to all algorithms
params = {
    # Stopping condition
    "stop_cond": "neval",
    "neval": 5e4,
    # Encoding
    "encoding": "real",
    # Disable verbose, the output gets very cluttered otherwise
    "verbose": False,
    # Number of times to launch each algorithm
    "repetitions": 10,
}

# Define the algorithms we are going to use
algorithms = [
    simple.random_search(rastrigin_func, params),
    simple.hill_climb(rastrigin_func, params),
    simple.simulated_annealing(rastrigin_func, params),
    simple.evolution_strategy(rastrigin_func, params),
    simple.genetic_algorithm(rastrigin_func, params),
    simple.differential_evolution(rastrigin_func, params),
    simple.particle_swarm(rastrigin_func, params),
]

# Instanciate the AlgorithmSelection class
algorithm_search = AlgorithmSelection(algorithms)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

Running 7 algorithms 10 times each.


The optimize function should have returned the best overall solution and a report with statistics about the execution

In [18]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [-5.62413067e-10 -1.61198807e-09]
fitness: 0.0


In [19]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
5,DE,1.520492,1.530608,1.550883,0.008116,1.520930,1.540948,1.591315,0.019540,0.000000,0.000000,0.000000,0.000000
6,PSO,1.114441,1.117173,1.119894,0.001864,1.115208,1.120089,1.123972,0.002533,0.000000,0.000000,0.000000,0.000000
0,RandomSearch,1.228865,1.234862,1.246519,0.005216,1.228153,1.236596,1.257287,0.008905,0.011010,0.043353,0.175698,0.052769
4,GA,1.332908,1.348921,1.364049,0.009361,1.332115,1.347973,1.363056,0.009303,0.362408,0.596018,1.141107,0.376147
3,ES,1.210424,1.223651,1.233134,0.007641,1.209707,1.222925,1.232482,0.007656,1.408577,1.855767,4.026313,0.959786
1,HillClimb,1.102137,1.113711,1.126980,0.008353,1.101563,1.116074,1.133981,0.010170,1.989918,1.989918,1.989918,0.000000
2,SA,1.108947,1.121849,1.133034,0.007770,1.109094,1.122825,1.133130,0.008283,17.327921,18.703447,23.615755,1.945337


With this, we can clearly see that the best algorithm for this task seems to be the Differential Evolution algorithm, and surprisingly, RandomSearch performs better than HillClimb, SA, ES and even GA. This could be caused by some misconfiguration of parameters, but since we just used the default implementation, we have no way of changing this.

## Strategy Selection
We could have specified the algorithms directly constructing them piece by piece, but unless you need to try some specific implementation of an algorithm, it is much simpler to search for the best Search Strategy. This is where the StrategySearch class is used. It is very similar to the AlgorithmSelection class, but using search strategies.

This also has the advantage that you can choose a custom name for each strategy.

In [25]:
# Initializer for sinlge-solution algorithms
single_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=1)

# Initializer for population-based algorithms
pop_initializer = UniformVectorInitializer(rastrigin_func.vecsize, rastrigin_func.low_lim, rastrigin_func.up_lim, pop_size=100)

# Define strategies to be tested
strategies = [
    RandomSearch(pop_initializer),
    HillClimb(
        single_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        name="HillClimb",
    ),
    SA(
        single_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="SA",
    ),
    SA(
        pop_initializer,
        perturb_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        params={"iter": 100, "temp_init": 1, "alpha": 0.995},
        name="ParallelSA",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 5e-3}),
        selection_op=SurvivorSelection("(m+n)"),
        params={"offspringSize": 150},
        name="ES-(μ+λ)",
    ),
    ES(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 5e-2}),
        selection_op=SurvivorSelection("(m,n)"),
        params={"offspringSize": 700},
        name="ES-(μ,λ)",
    ),
    GA(
        pop_initializer,
        mutation_op=OperatorReal("RandNoise", {"method": "Gauss", "F": 1e-3}),
        cross_op=OperatorReal("Multipoint"),
        parent_sel_op=ParentSelection("Tournament", {"amount": 60, "p": 0.1}),
        selection_op=SurvivorSelection("Elitism", {"amount": 10}),
        params={"pcross": 0.9, "pmut": 0.1},
        name="GA",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/best/1",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/rand/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/rand/1",
    ),
    DE(
        pop_initializer,
        OperatorReal("DE/current-to-best/1", {"F": 0.8, "Cr": 0.8}),
        name="DE/current-to-best/1",
    ),
    PSO(pop_initializer, {"w": 0.7, "c1": 1.5, "c2": 1.5}, name="PSO"),
]

# Instanciate the StrategySelection class
algorithm_search = StrategySelection(
    rastrigin_func,
    strategies,
    algorithm_params={
        "stop_cond": "neval",
        "neval": 5e4,
        "verbose": False,
    },
    params={"verbose": True, "repetitions": 10},
)

# Launch all the algorithms and get the best overall solution
solution, best_fitness, report = algorithm_search.optimize()

Running 11 algorithms 10 times each.


In [26]:
print(f"best solution: {solution}")
print(f"fitness: {best_fitness}")

best solution: [ 2.37988643e-09 -6.57376556e-11]
fitness: 0.0


In [27]:
report.sort_values("fitness_avg")

,name,realtime_min,realtime_avg,realtime_max,realtime_std,cputime_min,cputime_avg,cputime_max,cputime_std,fitness_min,fitness_avg,fitness_max,fitness_std
7,DE/best/1,1.517819,1.527340,1.548017,0.009033,1.516974,1.533311,1.550402,0.011164,0.000000e+00,0.000000,0.000000,0.000000
8,DE/rand/1,1.137253,1.143211,1.153556,0.006005,1.136587,1.147642,1.167720,0.010764,0.000000e+00,0.000000,0.000000,0.000000
9,DE/current-to-best/1,1.587128,1.605037,1.630802,0.011568,1.594514,1.616103,1.666758,0.024510,0.000000e+00,0.000000,0.000000,0.000000
10,PSO,1.111841,1.119677,1.131444,0.007129,1.113091,1.120147,1.130905,0.006818,0.000000e+00,0.000000,0.000000,0.000000
3,ParallelSA,0.897361,0.902998,0.907294,0.003329,0.897819,0.904131,0.908006,0.003450,2.479228e-06,0.000002,0.000002,0.000000
5,"ES-(μ,λ)",0.926531,0.941216,0.986965,0.017720,0.928907,0.946668,0.987425,0.017392,1.020093e-05,0.000010,0.000010,0.000000
0,RandomSearch,1.223030,1.239495,1.269671,0.016661,1.227257,1.243412,1.270551,0.017198,1.361403e-02,0.071581,0.350184,0.110165
6,GA,2.037897,2.059031,2.109529,0.019590,2.036398,2.056015,2.105052,0.019093,1.805098e-09,0.099496,0.994959,0.314634
4,ES-(μ+λ),0.921226,0.934589,0.954951,0.010694,0.924163,0.959393,1.066984,0.044430,9.949592e-01,1.193951,1.989918,0.419512
1,HillClimb,1.102126,1.114109,1.119963,0.005565,1.101476,1.113949,1.119386,0.005817,3.979831e+00,5.372768,17.909202,4.404854
